<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/image2image/VideoExtender__frame_interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

__Video Extender: Pytti-tools / frame-interpolation__  


_Credits:[DigThatData](https://twitter.com/DigThatData)
License: MIT_

---

<details>
<summary>
Example video
</summary>

![preview](https://i.imgur.com/NliA9Kj.gif)  
https://github.com/pytti-tools/frame-interpolation  
</details>

[![Buy me a beer](https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/assets/buy-me-a-beer.png ) ](https://steinhaug.com/donate/)

In [ ]:
import os
from pathlib import Path
from IPython.display import clear_output
def return__folderName(directory_path, verify_folder=False):
    if not verify_folder:
        return os.path.basename(os.path.normpath(directory_path))
    if os.path.isdir(directory_path):
        last_folder_name = os.path.basename(os.path.normpath(directory_path))
        return last_folder_name
    else:
        return None

drive_mounted = False
gdrive_fpath = '.'
local_path = '/content/'

##############################

#@markdown # 1. Setup Workspace

#@markdown Run this cell to perform setup.

#@markdown Mounting your google drive is optional.
#@markdown If you mount your drive, code and models will be downloaded to it.
#@markdown This should reduce setup time after your first run.

###################

mount_gdrive = True # @param{type:"boolean"}
if mount_gdrive and not drive_mounted:
    from google.colab import drive

    gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
    gdrive_subdirectory = 'MyDrive/interpolation' #@param{type:"string"}
    gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
    try:
        drive.mount(gdrive_mountpoint, force_remount = True)
        !mkdir -p {gdrive_fpath}
        %cd {gdrive_fpath}
        local_path = gdrive_fpath
        drive_mounted = True
    except OSError:
        print(
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            "If you received a scary OSError and your drive"
            " was already mounted, ignore it."
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            )
        raise

#####################

# Perform rest of setup

if not Path('./frame-interpolation').exists():
    !git clone https://github.com/pytti-tools/frame-interpolation

try:
    import frame_interpolation
except ModuleNotFoundError:
    %pip install -r ./frame-interpolation/requirements_colab.txt
    %pip install ./frame-interpolation

!pip install loguru
!pip install apache_beam
!pip install mediapy

#url = "https://drive.google.com/drive/folders/1GhVNBPq20X7eaMsesydQ774CgGcDGkc6?usp=sharing"
share_id = "1GhVNBPq20X7eaMsesydQ774CgGcDGkc6"

if not (Path(local_path) / 'saved_model').exists():
    !pip install --upgrade gdown
    !gdown --folder {share_id}

# create default frame
!mkdir -p frames

In [ ]:
# https://drive.google.com/drive/folders/1q8110-qp225asX3DQvZnfLfJPkCHmDpy?usp=sharing
share_id = "1q8110-qp225asX3DQvZnfLfJPkCHmDpy"
if not (Path(local_path) / 'pretrained_models').exists():
    !pip install --upgrade gdown
    !gdown --folder {share_id}
clear_output()
print('[1;32mDone! ✓')

In [45]:
#@markdown # 2. Setup Interpolation

#@markdown Specify the directory containing your video frames with the `frames_dir` parameter.

frames_dir = "/content/drive/MyDrive/output-comfyui/0" #@param{'type':'string'}

#@markdown Directory where interpolated video will be saved.

output_path = "/content/drive/MyDrive/output-comfyui" #@param{'type':'string'}

#@markdown A single pass of the interpolation procedure adds a frame between each contiguous pair of frames in `frames_dir`.

# If you start with $n$ frames in `frames_dir` and set `recursive_interpolation_passes` to $k$, your total number of frames after interpolation will be: 2^k (n-1) -1
recursive_interpolation_passes = 0 #@param {type: "number"}

# Frame rate of output video
output_video_fps = 30 #@param {type: "number"}

recursion_times = 6 #@param {type:"slider", min:1, max:10, step:1}

model_folder = "./pretrained_models/film_net/Style/saved_model" # @param ["./saved_model", "./pretrained_models/film_net/L1/saved_model", "./pretrained_models/film_net/Style/saved_model", "./pretrained_models/film_net/VGG/saved_model"]

#@markdown Check this box to generate a video output. If no output video will be generated, the FPS option can be ignored.
output_video = True #@param{'type':'boolean'}


In [46]:
#@markdown # 3. Interpolate and create video!
#@markdown Results will be saved in a output_path "interpolated_*.mp4"

import os.path
from loguru import logger

logger.info("Beginning interpolation...")

if output_video:

    #  --fps {output_video_fps} \
    #  --times_to_interpolate $recursion_times \

    !python -m frame_interpolation.eval.interpolator_cli \
      --model_path {model_folder} \
      --pattern {frames_dir} \
      --fps {output_video_fps} \
      --output_video

else:
    !python -m frame_interpolation.eval.interpolator_cli \
      --model_path {model_folder} \
      --pattern {frames_dir} \

clear_output()
logger.info("Interpolation complete.")

!cp {frames_dir}/interpolated.mp4 {output_path}
ip_id = return__folderName(frames_dir)
!mv {output_path}/interpolated.mp4 {output_path}/interpolated_{ip_id}.mp4
#!mv {output_path}/interpolated.mp4 {output_path}/interpolated_{ip_id}_{recursion_times}.mp4

if not os.path.exists(f"{output_path}/interpolated_{ip_id}.mp4"):
#if not os.path.exists(f"{output_path}/interpolated_{ip_id}_{recursion_times}.mp4"):
    raise Exception("Expected output file does not exist.")
else:
    print(f"Video saved at {output_path}/interpolated_{ip_id}.mp4")

2024-02-03 19:36:38.838 | INFO     | __main__:<cell line: 19>:19 - Interpolation complete.


Video saved at /content/drive/MyDrive/output-comfyui/interpolated_0.mp4


```
MIT License

Copyright (c) 2022 David Marx

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```